In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
from cs285.envs.pendulum.pendulum_env import PendulumEnv
from cs285.envs.dt_sampler import ConstantSampler, UniformSampler, ExponentialSampler
from cs285.infrastructure.replay_buffer import ReplayBufferTrajectories
from cs285.infrastructure.utils import sample_n_trajectories, RandomPolicy
from cs285.agents.ode_agent import ODEAgent
from cs285.agents.nueral_ode import Base_NeuralODE, NeuralODE_Vanilla, Pendulum_True_Dynamics, NeuralODE_Augmented, NeuralODE_Latent_MLP, ODE_RNN
from cs285.agents.utils import save_leaves, load_leaves
from cs285.infrastructure import utils
from typing import Callable, Optional, Tuple, Sequence
import numpy as np
import gym
from cs285.infrastructure import pytorch_util as ptu
from tqdm import trange
import jax
import jax.numpy as jnp
import equinox as eqx
import diffrax
from diffrax import diffeqsolve, Dopri5
import optax
import pickle
from tqdm import trange
import matplotlib.pyplot as plt

In [2]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

cpu


In [3]:
key = jax.random.PRNGKey(0)
dt_key, key = jax.random.split(key)
dt_sampler = ExponentialSampler(lmbda=20, key=dt_key)
env = PendulumEnv(
    dt_sampler=dt_sampler
)
mpc_dt_sampler = ConstantSampler(dt=0.05)
agent_key, key = jax.random.split(key)
neural_ode_name = "pendulum_true_dynamics"
neural_ode_kwargs = {
    "ode_dt0": 0.05,
}
optimizer_name = "adamw"
optimizer_kwargs = {"learning_rate": 1e-3}
mb_agent = ODEAgent(
    env=env,
    key=agent_key,
    neural_ode_name=neural_ode_name,
    neural_ode_kwargs=neural_ode_kwargs,
    optimizer_name=optimizer_name,
    optimizer_kwargs=optimizer_kwargs,
    ensemble_size=1,
    train_discount=1,
    mpc_horizon_steps=20,
    mpc_dt_sampler=mpc_dt_sampler,
    mpc_strategy="random",
    mpc_discount=0.95,
    mpc_num_action_sequences=1000,
    cem_num_iters=4,
    cem_num_elites=5,
    cem_alpha=1,
)
trajs, _ = sample_n_trajectories(env, mb_agent, ntraj=1000, max_length=200, key=key)
replay_buffer = ReplayBufferTrajectories()
replay_buffer.add_rollouts(trajs)
with open("true_dynamics_exponential_20", "wb") as f:
    pickle.dump(replay_buffer, f)

100%|██████████| 1000/1000 [1:45:37<00:00,  6.34s/it]


In [ ]:
key = jax.random.PRNGKey(0)
dt_key, key = jax.random.split(key)
dt_sampler = UniformSampler(high=0.1, low=0.0, key=dt_key)
env = PendulumEnv(
    dt_sampler=dt_sampler
)
mpc_dt_sampler = ConstantSampler(dt=0.05)
agent_key, key = jax.random.split(key)
neural_ode_name = "pendulum_true_dynamics"
neural_ode_kwargs = {
    "ode_dt0": 0.05,
}
optimizer_name = "adamw"
optimizer_kwargs = {"learning_rate": 1e-3}
mb_agent = ODEAgent(
    env=env,
    key=agent_key,
    neural_ode_name=neural_ode_name,
    neural_ode_kwargs=neural_ode_kwargs,
    optimizer_name=optimizer_name,
    optimizer_kwargs=optimizer_kwargs,
    ensemble_size=1,
    train_discount=1,
    mpc_horizon_steps=20,
    mpc_dt_sampler=mpc_dt_sampler,
    mpc_strategy="random",
    mpc_discount=0.95,
    mpc_num_action_sequences=1000,
    cem_num_iters=4,
    cem_num_elites=5,
    cem_alpha=1,
)
trajs, _ = sample_n_trajectories(env, mb_agent, ntraj=1000, max_length=200, key=key)
replay_buffer = ReplayBufferTrajectories()
replay_buffer.add_rollouts(trajs)
with open("true_dynamics_uniform_0_0.1", "wb") as f:
    pickle.dump(replay_buffer, f)

In [ ]:
key = jax.random.PRNGKey(0)
dt_sampler = ConstantSampler(dt=0.05)
env = PendulumEnv(
    dt_sampler=dt_sampler
)
mpc_dt_sampler = ConstantSampler(dt=0.05)
agent_key, key = jax.random.split(key)
neural_ode_name = "pendulum_true_dynamics"
neural_ode_kwargs = {
    "ode_dt0": 0.05,
}
optimizer_name = "adamw"
optimizer_kwargs = {"learning_rate": 1e-3}
mb_agent = ODEAgent(
    env=env,
    key=agent_key,
    neural_ode_name=neural_ode_name,
    neural_ode_kwargs=neural_ode_kwargs,
    optimizer_name=optimizer_name,
    optimizer_kwargs=optimizer_kwargs,
    ensemble_size=1,
    train_discount=1,
    mpc_horizon_steps=20,
    mpc_dt_sampler=mpc_dt_sampler,
    mpc_strategy="random",
    mpc_discount=0.95,
    mpc_num_action_sequences=1000,
    cem_num_iters=4,
    cem_num_elites=5,
    cem_alpha=1,
)
trajs, _ = sample_n_trajectories(env, mb_agent, ntraj=1000, max_length=200, key=key)
replay_buffer = ReplayBufferTrajectories()
replay_buffer.add_rollouts(trajs)
with open("true_dynamics_constant_dt", "wb") as f:
    pickle.dump(replay_buffer, f)